# Trainingspipeline 3

- ohne Bigramme
- vector_size: 300
- window: 5
- min_count: 2
- seed: 1
- CBOW

In [1]:
import codecs
import nltk
import numpy as np
import os
import pandas as pd
import re
import scipy
import spacy

from gensim.models import Word2Vec
from joblib import Parallel, delayed  
from nltk.corpus import stopwords

In [2]:
stopwords = stopwords.words('italian')
tokenizer = nltk.data.load('tokenizers/punkt/italian.pickle')

In [3]:
df = pd.read_csv('../../Korpus/Korpus/corpus_lemmatized.csv', sep=',', encoding='utf-8')

In [5]:
df.shape

(304129, 10)

In [6]:
df.text = df.text.fillna('')
df.lemmatized_text = df.lemmatized_text.fillna('')

In [7]:
# Einzeldataframes für die Zeiträume

df_periods = dict(tuple(df.groupby(by='period')))

In [8]:
df1 = df_periods['1700-1750']
df2 = df_periods['1751-1800']
df3 = df_periods['1801-1825']
df4 = df_periods['1826-1850']
df5 = df_periods['1851-1875']
df6 = df_periods['1876-1900']
df7 = df_periods['1901-1925']
df8 = df_periods['1926-1950']
df9 = df_periods['1951-1985']
df10 = df_periods['1986-2000']
df11 = df_periods['2001-2021']

## Training von Word2Vec

In [10]:
# Hilfsfunktionen zur Vorbereitung auf das Training
# Bereinigung und Tokenisierung

def sentence_to_wordlist(raw:str):
    """
    cleans and tokenizes the sentences
    """
    text = re.sub('[^A-Za-z_àÀèÈìÌòÒùÙáÁéÉíÍóÓúÚ]',' ', raw).split()        # Diakritika ans Italienische anpassen                    
    filtered_text = [word for word in text if word not in stopwords]        # Stopwörter löschen
    return filtered_text


def tokenize_text(raw_text):
    """
    returns a list of lowercase tokenized sentences 
    """
    raw_sentences = tokenizer.tokenize(str(raw_text).lower())    
    tokenized_sentences = Parallel(n_jobs=-1)(delayed(sentence_to_wordlist)(raw_sentence) for raw_sentence in raw_sentences)
    sentences = tokenized_sentences
    return sentences

In [11]:
# Trainingsparamter setzen

vector_size = 300                  # Dimensionality of the word vectors
window = 5                         # The maximum distance between the current and predicted word within a sentence
min_count = 2                      # (int, optional) – The model ignores all words with total frequency lower than this
workers = 1                        # Use these many worker threads to train the model (faster training with multicore machines)
min_alpha = 0.0001                 # Learning rate will linearly drop to min_alpha as training progresses
sg = 0                             # Training algorithm: skip-gram if sg=1, otherwise CBOW            
seed = 1                           # Reproductivity --> only if workers = 1

### Zeitraum 1: 1700-1750

In [12]:
# lemmatisierte Texte zu einem String verbinden

text1 = ''

for i in df1.lemmatized_text:
    text1 += i

In [13]:
%%time
sentences1 = tokenize_text(text1)         # Bereinigen, Tokenisieren und in Form bringen (Ziel: Liste von tokenisierten Sätzen)

Wall time: 18.1 s


In [14]:
print(sentences1[3])

['lungo', 'inospita', 'campagna', 'aggiungere', 'invidia', 'lato', 'manco', 'dire', 'anch', 'essere', 'teco', 'labbro', 'bianco', 'veggo', 'veneno', 'cor', 'stagno']


In [15]:
len(sentences1)

113328

In [16]:
%%time

# Training   

w2v1 = Word2Vec(sentences=sentences1,                      
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 34 s


In [17]:
w2v1.wv.most_similar(positive=['terrore'], topn=10)

[('vincitore', 0.9185842871665955),
 ('assalto', 0.9130616784095764),
 ('macello', 0.9021474719047546),
 ('fretta', 0.9002639055252075),
 ('desolazione', 0.8998203873634338),
 ('guasto', 0.8972659707069397),
 ('dietro', 0.8937568068504333),
 ('naviglio', 0.8909759521484375),
 ('distruggere', 0.8895993232727051),
 ('agitazione', 0.8871406316757202)]

In [18]:
# trainiertes Modell speichern

w2v1.save(os.path.join('trained_models/Word2VecCBOW2', '2w2vcbow1.model'))

### Zeitraum 2: 1751-1800

In [19]:
text2 = ''

for i in df2.lemmatized_text:
    text2 += i

In [20]:
%%time
sentences2 = tokenize_text(text2)

Wall time: 17.7 s


In [21]:
%%time
w2v2 = Word2Vec(sentences=sentences2,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 31.8 s


In [22]:
w2v2.wv.most_similar(positive=['terrore'], topn=10)

[('implacabil', 0.9620671272277832),
 ('atterrire', 0.960282564163208),
 ('schermo', 0.9601377844810486),
 ('inaspettato', 0.959879457950592),
 ('colmo', 0.9590060114860535),
 ('varco', 0.9588145613670349),
 ('crudeltà', 0.9585757255554199),
 ('maligno', 0.9571744799613953),
 ('insultare', 0.9569348096847534),
 ('invitto', 0.9564363956451416)]

In [23]:
w2v2.save(os.path.join('trained_models/Word2VecCBOW2', '2w2vcbow2.model'))

### Zeitraum 3: 1801-1825

In [24]:
text3 = ''

for i in df3.lemmatized_text:
    text3 += i

In [25]:
%%time
sentences3 = tokenize_text(text3)

Wall time: 15.5 s


In [26]:
%%time
w2v3 = Word2Vec(sentences=sentences3,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 32.5 s


In [27]:
w2v3.wv.most_similar(positive=['terrore'], topn=10)

[('orrore', 0.9473972320556641),
 ('crudele', 0.935994029045105),
 ('vendetta', 0.9351168870925903),
 ('cieco', 0.9325847625732422),
 ('ardore', 0.9306163787841797),
 ('infiammare', 0.9256657958030701),
 ('generoso', 0.9240584969520569),
 ('voluttà', 0.92402583360672),
 ('innocente', 0.9239453077316284),
 ('magnanimo', 0.9236640930175781)]

In [28]:
w2v3.save(os.path.join('trained_models/Word2VecCBOW2', '2w2vCBOW2.model'))

### Zeitraum 4: 1826-1850

In [29]:
text4 = ''

for i in df4.lemmatized_text:
    text4 += i

In [30]:
%%time
sentences4 = tokenize_text(text4)

Wall time: 15.7 s


In [31]:
%%time
w2v4 = Word2Vec(sentences=sentences4,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 30.6 s


In [32]:
w2v4.wv.most_similar(positive=['terrore'], topn=10)

[('nutrire', 0.9233593940734863),
 ('vigore', 0.9211021661758423),
 ('frode', 0.9208558797836304),
 ('senno', 0.9160754084587097),
 ('virt', 0.912816047668457),
 ('cosmopolitiche', 0.9115775227546692),
 ('ardore', 0.9112491011619568),
 ('intelletto', 0.9111162424087524),
 ('odio', 0.9110170602798462),
 ('diffidenza', 0.9103611707687378)]

In [33]:
w2v4.save(os.path.join('trained_models/Word2VecCBOW2', '2w2vcbow4.model'))

### Zeitraum 5: 1851-1875

In [34]:
text5 = ''

for i in df5.lemmatized_text:
    text5 += i

In [35]:
%%time
sentences5 = tokenize_text(text5)

Wall time: 16.5 s


In [36]:
%%time
w2v5 = Word2Vec(sentences=sentences5,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 31.5 s


In [37]:
w2v5.wv.most_similar(positive=['terrore'], topn=10)

[('raccapriccio', 0.9193395376205444),
 ('ansia', 0.9192577004432678),
 ('angoscia', 0.9088446497917175),
 ('spavento', 0.9064421653747559),
 ('commozione', 0.901406466960907),
 ('impeto', 0.8971648216247559),
 ('accoramento', 0.895581841468811),
 ('ira', 0.8930152058601379),
 ('amarezza', 0.892980694770813),
 ('sgomento', 0.8906183838844299)]

In [38]:
w2v5.save(os.path.join('trained_models/Word2VecCBOW2', '2w2vcbow5.model'))

### Zeitraum 6: 1876-1900

In [39]:
text6 = ''

for i in df6.lemmatized_text:
    text6 += i

In [40]:
%%time
sentences6 = tokenize_text(text6)

Wall time: 19.5 s


In [41]:
%%time
w2v6 = Word2Vec(sentences=sentences6,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 30.6 s


In [42]:
w2v6.wv.most_similar(positive=['terrore'], topn=10)

[('turbamento', 0.9090649485588074),
 ('palpito', 0.9023292660713196),
 ('spasimo', 0.8985578417778015),
 ('sgomento', 0.8970476984977722),
 ('inestinguibile', 0.8931776881217957),
 ('violento', 0.8922560811042786),
 ('molce', 0.8912482857704163),
 ('ardente', 0.8893857002258301),
 ('tristezza', 0.8887675404548645),
 ('lamento', 0.8877881169319153)]

In [43]:
w2v6.save(os.path.join('trained_models/Word2VecCBOW2', '2w2vcbow6.model'))

### Zeitraum 7: 1901-1925

In [44]:
text7 = ''

for i in df7.lemmatized_text:
    text7 += i

In [45]:
%%time
sentences7 = tokenize_text(text7)

Wall time: 19 s


In [46]:
%%time
w2v7 = Word2Vec(sentences=sentences7,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 31.4 s


In [47]:
w2v7.wv.most_similar(positive=['terrore'], topn=10)

[('angoscia', 0.9372536540031433),
 ('sgomento', 0.9344909191131592),
 ('spavento', 0.9292963147163391),
 ('commozione', 0.9200077652931213),
 ('gioia', 0.9193939566612244),
 ('ira', 0.905076265335083),
 ('dolcezza', 0.9032096266746521),
 ('stupore', 0.9024162888526917),
 ('impeto', 0.9020339846611023),
 ('tenerezza', 0.8951994776725769)]

In [48]:
w2v7.save(os.path.join('trained_models/Word2VecCBOW2', '2w2vcbow7.model'))

### Zeitraum 8: 1926-1950

In [49]:
text8 = ''

for i in df8.lemmatized_text:
    text8 += i

In [50]:
%%time
sentences8 = tokenize_text(text8)

Wall time: 17.7 s


In [51]:
%%time
w2v8 = Word2Vec(sentences=sentences8,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 30.7 s


In [52]:
w2v8.wv.most_similar(positive=['terrore'], topn=10)

[('spavento', 0.9505687952041626),
 ('sgomento', 0.939396858215332),
 ('grido', 0.9114935398101807),
 ('lacrima', 0.9089455008506775),
 ('angoscia', 0.9070513248443604),
 ('bacio', 0.8922446370124817),
 ('tremare', 0.8887932300567627),
 ('ansia', 0.8879618644714355),
 ('pianto', 0.883679211139679),
 ('freddo', 0.8821359276771545)]

In [53]:
w2v8.save(os.path.join('trained_models/Word2VecCBOW2', '2w2vcbow8.model'))

### Zeitraum 9: 1951-1985

In [54]:
text9 = ''

for i in df9.lemmatized_text:
    text9 += i

In [55]:
%%time
sentences9 = tokenize_text(text9)

Wall time: 19.3 s


In [56]:
%%time
w2v9 = Word2Vec(sentences=sentences9,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 32.1 s


In [57]:
w2v9.wv.most_similar(positive=['terrore'], topn=10)

[('disperazione', 0.9268671870231628),
 ('aldilà', 0.9091119170188904),
 ('disperato', 0.9062608480453491),
 ('odio', 0.8979014158248901),
 ('celeste', 0.8873206377029419),
 ('sacerdotessa', 0.8871921300888062),
 ('ira', 0.8821128010749817),
 ('velo', 0.8789607286453247),
 ('impietosire', 0.877223789691925),
 ('solitudine', 0.8757528066635132)]

In [58]:
w2v9.save(os.path.join('trained_models/Word2VecCBOW2', '2w2vcbow9.model'))

### Zeitraum 10: 1986-2000

In [59]:
text10 = ''

for i in df10.lemmatized_text:
    text10+= i

In [60]:
%%time
sentences10 = tokenize_text(text10)

Wall time: 35 s


In [61]:
%%time
w2v10 = Word2Vec(sentences=sentences10,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 30.8 s


In [62]:
w2v10.wv.most_similar(positive=['terrore'], topn=10)

[('mutilare', 0.9516743421554565),
 ('buio', 0.9414613842964172),
 ('bel', 0.9318793416023254),
 ('sfigurato', 0.9254973530769348),
 ('giallo', 0.9253699779510498),
 ('sangue', 0.9242494106292725),
 ('freddo', 0.9241604804992676),
 ('dicitura', 0.9240567088127136),
 ('bimbo', 0.9227294325828552),
 ('orrendamente', 0.9212974905967712)]

In [63]:
w2v10.save(os.path.join('trained_models/Word2VecCBOW2', '2w2vcbow10.model'))

### Zeitraum 11: 2001-2021

In [64]:
text11 = ''

for i in df11.lemmatized_text:
    text11+= i

In [65]:
%%time
sentences11 = tokenize_text(text11)

Wall time: 12.5 s


In [66]:
%%time
w2v11 = Word2Vec(sentences=sentences11,                   
                vector_size=vector_size,          
                window=window,                
                min_count=min_count,              
                workers=workers, 
                min_alpha=min_alpha,         
                sg=sg,                     
                seed=seed)

Wall time: 31.9 s


In [67]:
w2v11.wv.most_similar(positive=['terrore'], topn=10)

[('tristezza', 0.9893108010292053),
 ('sgomentare', 0.9882932901382446),
 ('eterno', 0.987527072429657),
 ('disperazione', 0.9865819811820984),
 ('variconi', 0.9863195419311523),
 ('malo', 0.9858474135398865),
 ('orecchio', 0.9815552234649658),
 ('curioso', 0.9814350605010986),
 ('solitudine', 0.9813984632492065),
 ('mostro', 0.9809895753860474)]

In [68]:
w2v11.save(os.path.join('trained_models/Word2VecCBOW2', '2w2vcbow11.model'))